In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
import tensorflow as tf


In [130]:
x_train =pd.read_csv('../input/sign-language-mnist/sign_mnist_train/sign_mnist_train.csv')
x_test=pd.read_csv('../input/sign-language-mnist/sign_mnist_test/sign_mnist_test.csv')


In [131]:
y_train=x_train['label']
y_test=x_test['label']
x_train=x_train.drop(columns = ['label'])
x_test=x_test.drop(columns = ['label'])


In [100]:
"""""
from imblearn.over_sampling import RandomOverSampler 
oversample = RandomOverSampler()
x_train_sample,y_train_sample  = oversample.fit_resample(x_train,y_train)
"""""

In [101]:
#x_test_sample,y_test_sample  = oversample.fit_resample(x_test,y_test)

In [132]:
"""""
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

x_train_normalized = scaler.fit_transform(x_train)
x_test_normalized = scaler.fit_transform(x_test)
"""""
x_train_normalized = np.array(x_train)/255
x_test_normalized = np.array(x_test)/255

In [133]:
x_train_reshaped= x_train_normalized.reshape(-1,28,28,1)

In [134]:
x_test_reshaped= x_test_normalized.reshape(-1,28,28,1)

In [105]:
#y_train= np.array(y_train_sample) 
#y_test=np.array(y_test_sample)

In [135]:
model=Sequential()

model.add(Conv2D(128, kernel_size = (3,3), input_shape = (28, 28, 1), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(64, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.2))
model.add(Conv2D(32, kernel_size = (3,3), activation = 'relu'))
model.add(MaxPool2D(pool_size = (2,2)))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(64, activation='relu'))
model.add(Dense(25, activation='softmax'))

In [136]:

model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

checkpoint_filepath = 'sign_best_weights.h'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_weights_only=True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

model.fit(x_train_reshaped,y_train,validation_split=0.2,batch_size = 100,epochs = 50,callbacks=[model_checkpoint_callback])

In [137]:
model.evaluate(x_test_reshaped, y_test)